In [1]:
import pickle
import pandas as pd
import os
import sys
sys.path
from scipy import stats
sys.path.append('../simulation')
import TS_personal_params_pooled as pp
import TS_global_params_pooled as gtp
import numpy as np

In [3]:
with open('../../Downloads/big_mellongf.pkl','rb') as f:
    test = pickle.load(f)

In [4]:
def get_decision_times_sims(d,title,save_title,save=False,i=None,ax=None):
    actions = {}
    regrets ={}
    oregrets ={}
    steps = []
    for sim,sims in d['actions'].items():
        for k,v in sims.items():
            if k not in actions:
                actions[k]=[]
            actions[k].extend(v)
    for sim,sims in d['regrets'].items():
        for k,v in sims.items():
            if k not in regrets:
                regrets[k]=[]
            regrets[k].extend(v)
            
    for sim,sims in d['oregrets'].items():
        for k,v in sims.items():
            if k not in oregrets:
                oregrets[k]=[]
            oregrets[k].extend(v)
   # for sim,sims in d['steps'].items():
        
           
             
        #steps[k].extend(sims)
 
    return get_to_plot_actions(actions),get_to_plot_regrets(regrets),steps,get_to_plot_regrets(oregrets)
#,get_to_plot_regrets(steps)

In [7]:
def get_to_plot_actions(actions):
    
    skeys = sorted(actions.keys())
    to_plot = []
    for k in skeys:
        to_plot.append(sum(actions[k])/len(actions[k]))
    return to_plot
def get_to_plot_regrets(actions):
    
    skeys = sorted(actions.keys())
    to_plot = []
    for k in skeys:
        to_plot.append(np.array(actions[k]).mean())
    return to_plot

In [8]:
def get_all_d(dic):
    to_return = {}
    cases = ['case_one','case_two','case_three']
   
    for i in range(len(cases)):
        case = cases[i]
        d = dic[case]
        act,regret,steps,oregrets  = get_decision_times_sims(d,case,case,save=False,i=i,ax=None)
        to_return[case]={'actions':act,'regrets':regret,'steps':steps,'oregrets':oregrets}
    return to_return

In [9]:
t= get_all_d(test)

In [15]:
def get_weekly_averages(points):
    to_return = []
    sems = []
    for i in range(0,len(points)-5,35):
        temp = points[i:i+35]
        to_return.append(np.array(temp).mean())
        #print(temp)
        sems.append(stats.sem(temp))
    return to_return,sems

In [21]:
def get_decision_times(d,title,save_title,save=False,i=None,ax=None):
    
    x =get_to_plot_actions(d['actions'])
   
    to_plotr =  get_to_plot_regrets(d['regrets'])
    to_plotro =  get_to_plot_regrets(d['oregrets'])
    return x,to_plotr,[], to_plotro 

In [23]:
def get_results(results_path,pop_size,update_time,case,phrase,ophrase):
    
    
    files = os.listdir('{}{}'.format(results_path,case))
    
    steps = []

    #check population and pop_size,update_time
    files = [f for f in files if '.DS' not in f and pop_size in f and phrase in f and ophrase not in f and 'update_days_{}'.format(update_time) in f  and '2015' not in f    ]
    #files = files[:50]
    #print(len(files))
    #print(files)
    seen = 0 
    regrets = {}
    actions = {}
    oregrets = {}
    mus = {}
    for ff in files:
        #sim = int(ff.split('sim_')[1].strip('_continuous_pretreatment_base_coef_scale_hybrid.pkl'))
        #try:
        with open('{}{}/{}'.format(results_path,case,ff),'rb') as f:
            d = pickle.load(f)
            #return d
            #print(d.keys())
            if len(d)>0:
            #if int(sim)<60:
                #print(len([k['steps'] for k in d['history'].values() if k['avail'] and k['decision_time']]))
                steps = steps+[k['steps'] for k in d['history'].values() if k['avail'] and k['decision_time']]
                for k,v in d['actions'].items():
                    if k not in actions:
                        actions[k]=[]
                    actions[k].extend(v)
                for k,v in d['regrets'].items():
                    if k not in regrets:
                        regrets[k]=[]
                    regrets[k].extend(v)
                for k,v in d['oregrets'].items():
                    if k not in oregrets:
                        oregrets[k]=[]
                    oregrets[k].extend(v)
            seen = seen+1
           # print(ff)
    print(seen)
    return {'steps':steps,'oregrets':oregrets,'regrets':regrets,'actions':actions}

In [24]:
def get_all(results_path,pop,update,phrase,ophrase):
    to_return = {}
    cases = ['case_one','case_two','case_three']
   
    for i in range(len(cases)):
        case = cases[i]
        d = get_results(results_path,str(pop),update,cases[i],phrase,ophrase)
        
        act,regret,steps,oregrets = get_decision_times(d,case,case,save=False,i=i,ax=None)
        to_return[case]={'actions':act,'regrets':regret,'steps':steps,'oregrets':oregrets}
    return to_return

In [25]:
fb = get_all('../../Downloads/pooling_results_testing/batch/','32',7,'burden818twotimesaveshortmajoritymellong','?')



50
50
50


In [28]:
get_weekly_averages(fb['case_three']['actions'])

([0.5004,
  0.5651093604807892,
  0.6141931259683361,
  0.6052622960507966,
  0.6136312043599131,
  0.6467063949434144,
  0.6395018984571198,
  0.6176297380445297,
  0.6406791661754395,
  0.6185821670393098,
  0.6080448979591837,
  0.662312925170068],
 [0.006938448877748662,
  0.016791508326319777,
  0.014264405579163865,
  0.009764720668142053,
  0.009871089953265634,
  0.008915440870502184,
  0.007538973542658079,
  0.009256297049609438,
  0.011625707959425179,
  0.01469278041626629,
  0.02326901318960073,
  0.020716133661709196])

In [30]:
get_weekly_averages(t['case_two']['actions'])

([0.49484761904761904,
  0.5323557902415046,
  0.5991257555653839,
  0.5941422170768315,
  0.531570558841044,
  0.5904880330776624,
  0.5513184469838709,
  0.5147270426556733,
  0.5581168129154975,
  0.5937401138543995,
  0.5956075448361163,
  0.6669183673469388],
 [0.005688758105906334,
  0.0035982785289269043,
  0.008680137626781475,
  0.006273704763709249,
  0.013232082765799052,
  0.010733054896207633,
  0.014030770285323579,
  0.01594269750401586,
  0.015170618115134179,
  0.021215057089085172,
  0.02776854561707393,
  0.026428785824915])